# Median of two sorted lists redo

## Setup

### Imports

In [1]:
import numpy as np
import pandas as pd
from IPython.display import Markdown
from rich.jupyter import print

### Utils

```python 
def check_corner_cases(a, b, target_index):
    # scenarios 1 - 3
    if len(a) == 0 or (len(a) <= target_index and a[-1] <= b[0]):
        # scenario 3
        c = np.concat((a, b))
        median_input_values = (c[target_index - 1], c[target_index])
    elif len(a) > target_index and a[target_index] <= b[0]:
        # scenario 3
        median_input_values = (a[target_index - 1], a[target_index])
    elif len(b) > target_index and b[target_index] <= a[0]:
        # scenario 2
        median_input_values = (b[target_index - 1], b[target_index])
    else:
        median_input_values = False
        
    return median_input_values

def check_corner_cases(a, b, target_index):
    scenario_1 = len(a) == 0 or len(b) == 0 or a[-1] <= b[0]
    scenario_2 = b[-1] <= a[0]
    scenario_3 = len(a) < target_index and a[-1] < b[target_index - len(a) - 1]
    scenario_4

def calculate_median(median_input_values, n):
    #print(median_input_values)
    if n % 2 == 0:
        med = np.mean(median_input_values)
    else:
        med = median_input_values[1]
        
    return med

def median_search(a, b, pointers, target_index):
    low, a_split, high = pointers
    b_split = target_index - a_split
    
    #print(f"len(a): {len(a)}, target_index: {target_index}, a_split: {a_split}, b_split: {b_split}")
    #display(pd.Series(a).pipe(highlight_series, a_split, "lightblue"))
    #display(pd.Series(b).pipe(highlight_series, b_split, "lightgreen"))
    #display(Markdown("---"))
    
    a_lower_lte_b_upper = a[a_split - 1] <= b[b_split]
    b_lower_lte_a_upper = b[b_split - 1] <= a[a_split]
    
    if not a_lower_lte_b_upper:
        new_split = (low + a_split) // 2
        median_values = median_search(a, b, (low, new_split, a_split), target_index)
    elif not b_lower_lte_a_upper:
        new_split = (a_split + high) // 2
        adjustment = 1 if new_split == a_split else 0
        new_split += adjustment
        median_values = median_search(a, b, (a_split, new_split, high), target_index)
    else:
        return (max(a[a_split - 1], b[b_split - 1]), min(a[a_split], b[b_split]))
    return median_values


def main(list1, list2):
    a, b = (list1, list2) if len(list1) <= len(list2) else (list2, list1)

    na = len(a)
    n = na + len(b)
    target_index = n // 2
    
    #median_input_values = check_corner_cases(a, b, target_index)
    median_input_values = False
    #print(median_input_values)
    if not median_input_values:
        pointers = (0, na // 2, na)
        median_input_values = median_search(a, b, pointers, target_index)

    med = calculate_median(median_input_values, n)
    return float(med)

```

In [2]:
def highlight_greaterthan(value, threshold, color='yellow'):
    """Highlights cells greater than a threshold value."""
    if isinstance(value, (int, float)) and value >= threshold:
        return f'background-color: {color}'
    return ''

def highlight_series(series, idx, color="green"):
    cols = range(idx, len(series))
    return (
        series
        .to_frame()
        .T
        .style
        #.map(lambda x: highlight_greaterthan(x, series[idx], color))
        .set_properties(subset=[*cols], **{"background-color": color})
    )

def highlight_ol(ol, c="green"):
    cols = list(range(ol.search_idx_target, ol.n))
    s = pd.Series(ol.list).to_frame().T
    return s.style.set_properties(subset=cols, **{"background-color": c})

In [3]:
float("inf") > 8

True

In [20]:
class OrderedList(list):
    def __init__(self, nums, target=None):
        self.list = nums
        self.n = len(nums)
        self.search_idx_low = 0
        self.search_idx_target = self.n // 2 if target is None else target
        self.search_idx_high = self.n
        self.max_low = self.list[self.search_idx_target - 1]
        self.min_high = self.list[self.search_idx_target]

        # special cases
        if self.n == 0:
            self.search_idx_target = -1
            self.max_low = -float("inf")
            self.min_high = float("inf")
        elif self.search_idx_target == 0:
            self.max_low = -float("inf")

    def get_next_index(self, up=True):
        current_index = self.search_idx_target
        
        if up:
            next_index = (self.search_idx_target + self.search_idx_high) // 2
        else:
            next_index = (self.search_idx_target + self.search_idx_low) // 2

        if current_index == next_index:
            if next_index in [0, self.n - 1]:
                status = "over" if up else "under"
            else:
                status = "internal"
        else:
            status = "found next"

        return dict(next_index=next_index, status=status)
        
    def search(self, up=True):
        current_target = self.search_idx_target
        next_target = self.get_next_index(up)
        target = next_target["next_index"]
        status = next_target["status"]
        #print(status)

        if up: 
            if status == "over":
                self.max_low = self.list[-1]
                self.min_high = float("inf")
                self.search_idx_target = target + 1
            else:
                self.max_low = self.list[self.search_idx_target - 1]
                self.min_high = self.list[self.search_idx_target]
                self.search_idx_target = target
            self.search_idx_low = current_target
        else:
            if status == "under":
                self.max_low = -float("inf")
                self.min_high = self.list[0]
            else:
                self.search_idx_target = target
                self.search_idx_high = current_target
                low_idx = self.search_idx_target - 1
                self.max_low = self.list[low_idx] if low_idx >= 0 else -float("inf")
                self.min_high = self.list[self.search_idx_target]

        return status

    @property
    def search_range(self):
        return (self.search_idx_low, self.search_idx_target, self.search_idx_high)

    @property
    def border_values(self):
        return (self.max_low, self.min_high)

    def set_target(self, target):
        self.search_idx_target = target
        if target == 0:
            self.max_low = -float("inf")
            self.min_high = self.list[0]
        elif target >= self.n:
            self.max_low = self.list[-1]
            self.min_high = float("inf")
        else:
            self.max_low = self.list[target - 1]
            self.min_high = self.list[target]

    def __repr__(self):
        return str(self.list)

def calculate_median(median_input_values, n):
    #print(median_input_values)
    if n % 2 == 0:
        med = np.mean(median_input_values)
    else:
        med = median_input_values[1]
        
    return med
    
def main(a, b):
    a, b = (a, b) if len(a) <= len(b) else (b, a)
    a = OrderedList(a)
    b = OrderedList(b)
    target_index = (a.n + b.n) // 2
    b.set_target(target_index - a.search_idx_target)
    a, b = median_search(a, b, target_index)
    median_input_values = (
        max([i.max_low for i in [a, b]]), 
        min([i.min_high for i in [a, b]]),
    )
        
    display(highlight_ol(a, "lightgreen"))
    display(highlight_ol(b, "lightblue"))
    display(Markdown("---"))
    #print(median_input_values)
    med = calculate_median(median_input_values, a.n + b.n)
    return med
    
def median_search(a, b, target_index):
    a_lower_lte_b_upper = a.max_low <= b.min_high
    b_lower_lte_a_upper = b.max_low <= a.min_high
    
    if not a_lower_lte_b_upper:
        status = a.search(up=False)
        #if status == "found next":
        b.set_target(target_index - a.search_idx_target)
        a, b = median_search(a, b, target_index)
    elif not b_lower_lte_a_upper:
        status = a.search(up=True)
        #if status == "found next":
        b.set_target(target_index - a.search_idx_target)
        a, b = median_search(a, b, target_index)
    else:
        return a, b
        
    return a, b

In [21]:
x = OrderedList(list(range(10)))
#x.set_target(4)
print(f"pointers: {x.search_range}")
print(f"border: {x.border_values}")
directions = []
directions += [True] * 5
#directions += [False] * 5
#directions += [True, False] * 5
for d in directions:
    print(f"{'up' if d else 'down'} {'-'*100}")
    x.search(d)
    print(f"pointers: {x.search_range}")
    print(f"border: {x.border_values}")
    

pointers: (0, 5, 10)

border: (4, 5)

up ----------------------------------------------------------------------------------------------------

pointers: (5, 7, 10)

border: (4, 5)

...

up ----------------------------------------------------------------------------------------------------

pointers: (9, 10, 10)

border: (9, inf)

up ----------------------------------------------------------------------------------------------------

IndexError: list index out of range

## Givens

Two sorted integer series, a and b.  
Without loss of generality, len(a) $\leq$ len(b).  
Find the median of the combined series, c.

notation:
- a_lower = values in 'a' lower than the median
- a_upper = values in 'a' higher than the median
- likewise b_lower, b_upper, c_lower, c_upper



## Solution

Given a combined length of n we are looking for the index (0 indexed):  
- n // 2 if n is odd -> c[n // 2]
- (n // 2 - 1, n // 2) if n is even -> c[n // 2 - 1, n // 2]

Secenarios:
1. **len(a) == 0 OR len(a) < n // 2 and a[-1] < b[0]**  
   requires c_lower $\subseteq (a \cup b[ :j ])$  
   where j = n // 2 - len(a)  
   $\rightarrow$ c[n // 2 - 1, n // 2] = b[j - 1, j]  
   (*) len(b) < n // 2 is not possible since we ensure len(a) $\leq$ len(b)
1. **a[n // 2] < b[0]**  
  requires c_lower $\subseteq$ a  
  $\rightarrow$ c[n // 2 - 1, n // 2] = a[n // 2 - 1, n // 2]
1. **b[n // 2] < a[0]**  
  requires c_lower $\subseteq$ b  
  $\rightarrow$ c[n // 2 - 1, n // 2] = b[n // 2 - 1, n // 2]
1. **c_lower $\subseteq (a \cup b)$**  
  requires searching for the correct combination of splits in a and b such that c_lower $=$ (a_lower $\cup$ b_lower)

Scenarios:  

$n_a = |a|$  
$n_b = |b|$  
$n = n_a + n_b$  
$i = n // 2 - 1$  
$j = n // 2$  

1. $n_a == 0 \lor n_b == 0 \lor a \leq b$  
  $\implies c\:[i, j] = a\,b\:[i, j]$  
  ex: $a = [1, 2, 3],\quad b = [4, 5, 6],\quad c\:[i, j] = [3, 4]$
1. $b \leq a$  
  $\implies c\:[i, j] = b\,a\:[i, j]$
1. $n_a < j \land a \leq b\,[n_b - j]$  
  $\implies c\:[i, j] = a\,b\:[i, j]$  
  ex: $a = [1, 3],\quad b = [2, 4, 5, 6, 7, 8],\quad c\:[i, j] = [4, 5]$
1. $b\:[j] < a$  
  $\implies c_{lower} \subseteq b$  
  $\implies c\:[i, j] = b\:[i, j]$  
  ex: $a = [5],\quad b = [1, 2, 3, 4, 6],\quad c\:[i, j] = [3, 4]$  
  (*) $a\:[j] < b$ only happens when $|a| = j \implies a \leq b$  
1. **c_lower $\subseteq (a \cup b)$**  
  requires searching for the correct combination of splits in a and b such that c_lower $=$ (a_lower $\cup$ b_lower)

search algo (3):
- split a into a_lower and a_upper at index i
- split b into b_lower and b_lower at index j, where j = (n // 2 - i)
- until a[i - 1] $\leq$ b[j] and b[j - 1] $\leq$ a[i] # (a_lower $\cup$ b_lower) = c_lower    
  if a[i - 1] $\gt$ b[j] # this means a_lower $\subseteq$ (c_lower $\cup$ c_upper):  
  $\rightarrow$ do a binary search of a_lower  
  elif a[i - 1] $\lt$ b[j] # this means c_lower $\subseteq$ (a_lower $\cup$ a_upper):  
  $\rightarrow$ do a binary search of a_upper  
  
  

## Example

In [24]:
override_a = False
override_b = False

na_ = np.random.randint(1, 100)
nb_ = np.random.randint(1, 100)

a_ = np.random.randint(0, 60, na_).tolist()
a_.sort()

#override_a = True
if override_a:
    a_ = [1, 2, 3]
    na_ = len(a_)

b_ = np.random.randint(40, 100, nb_).tolist()
b_.sort()

#override_b = True
if override_b:
    b_ = [5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
    nb_ = len(b_)

c_ = np.concat([a_, b_]).astype(int).tolist()
c_.sort()
n_ = len(c_)
med_calc = np.median(c_)

display(pd.Series(a_).pipe(highlight_series, na_ // 2, "lightblue"))
display(pd.Series(b_).pipe(highlight_series, nb_ // 2, "lightgreen"))

print(f"calculated median = {med_calc}")
display(pd.Series(c_).pipe(highlight_series, n_ // 2, "lightblue"))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92
0,0,1,1,1,1,2,4,4,5,6,7,7,7,8,9,10,11,12,12,13,13,13,14,14,15,16,17,17,19,19,21,21,22,22,22,22,24,24,25,25,25,26,26,27,30,31,32,32,33,33,34,34,34,34,34,35,36,37,38,40,40,40,41,43,43,43,44,45,45,46,46,48,48,50,50,51,51,52,52,53,53,53,53,54,54,54,55,55,56,57,59,59,59


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95
0,40,41,42,42,42,45,45,45,46,46,47,48,48,49,51,51,53,54,55,55,56,56,57,58,58,59,59,60,60,61,62,63,63,63,64,64,65,65,65,65,67,67,69,69,71,72,72,72,73,73,75,76,77,77,78,78,78,79,80,81,81,82,82,83,83,83,83,84,85,85,86,87,88,88,88,88,88,89,89,89,91,92,92,92,93,93,93,94,94,94,95,95,96,96,96,99


calculated median = 52.0

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188
0,0,1,1,1,1,2,4,4,5,6,7,7,7,8,9,10,11,12,12,13,13,13,14,14,15,16,17,17,19,19,21,21,22,22,22,22,24,24,25,25,25,26,26,27,30,31,32,32,33,33,34,34,34,34,34,35,36,37,38,40,40,40,40,41,41,42,42,42,43,43,43,44,45,45,45,45,45,46,46,46,46,47,48,48,48,48,49,50,50,51,51,51,51,52,52,53,53,53,53,53,54,54,54,54,55,55,55,55,56,56,56,57,57,58,58,59,59,59,59,59,60,60,61,62,63,63,63,64,64,65,65,65,65,67,67,69,69,71,72,72,72,73,73,75,76,77,77,78,78,78,79,80,81,81,82,82,83,83,83,83,84,85,85,86,87,88,88,88,88,88,89,89,89,91,92,92,92,93,93,93,94,94,94,95,95,96,96,96,99


In [25]:
med_algo = main(a_, b_)
print(
    f"{med_algo == med_calc}: "
    f"algo median ({med_algo}) == calculated median ({med_calc})"
)

RecursionError: maximum recursion depth exceeded

In [ ]:
def upper(split):
    return (split[1], (split[1] + split[2]) // 2, split[2])
    
def lower(split):
    return (split[0], (split[0] + split[1]) // 2, split[1])
    
x = [1, 2, 3, 4, 5, 6, 7, 8, 9]
split = 0, len(x) // 2, len(x)
print(split)
display(pd.Series(x).pipe(highlight_series, split[1], "lightgreen"))

In [ ]:
split = lower(split)
print(split)
display(pd.Series(x).pipe(highlight_series, split[1], "lightgreen"))

In [ ]:
split = upper(split)
print(split)
display(pd.Series(x).pipe(highlight_series, split[1], "lightgreen"))

In [ ]:
display(pd.Series(x).pipe(highlight_series, split[1] + 1, "lightgreen"))

In [ ]:
calculate_median((2, 3), 3)